In [ ]:
#!pip install opencv-python numpy matplotlib transformers torch pillow

In [12]:
import sys

import matplotlib
matplotlib.use('TkAgg') 
import matplotlib.pyplot as plt
%matplotlib tk

import cv2
import numpy as np
from matplotlib.patches import Polygon
from transformers import pipeline
from PIL import Image
import torch
import math

In [16]:
CAMINHO_IMAGEM = "foto_1_atual.png"

In [17]:
class DeepVolumeEstimator:
    def __init__(self, image_path):
        self.image_path = image_path
        self.img_cv2 = cv2.imread(image_path)
        
        if self.img_cv2 is None:
            raise FileNotFoundError(f"Não foi possível ler a imagem em: {image_path}")

        self.img_rgb = cv2.cvtColor(self.img_cv2, cv2.COLOR_BGR2RGB)
        self.pil_image = Image.fromarray(self.img_rgb)
        self.depth_map = None
        self.mask = None
        self.pixel_scale_m = None  # Inicializa como None para forçar a calibração

        print("Carregando modelo Depth Anything V2... (aguarde)")
        device = 0 if torch.cuda.is_available() else -1
        print(f"Usando dispositivo: {'GPU' if device == 0 else 'CPU'}")
        
        try:
            self.pipe = pipeline(task="depth-estimation", model="depth-anything/Depth-Anything-V2-Small-hf", device=device)
        except Exception as e:
            raise RuntimeError(f"Erro ao carregar modelo: {e}")

    def gerar_mapa_profundidade(self):
        print("Gerando mapa de profundidade...")
        output = self.pipe(self.pil_image)
        depth_pil = output["depth"]
        depth_np = np.array(depth_pil, dtype=np.float32)
        
        # Inverter para que 'perto/alto' seja 1.0
        self.depth_map = 1.0 - (depth_np - depth_np.min()) / (depth_np.max() - depth_np.min())
        
        plt.figure(figsize=(10, 5))
        plt.imshow(self.depth_map, cmap='magma')
        plt.title("Mapa de Elevação Relativa (Preview)")
        plt.colorbar(label="Altura Relativa")
        plt.show()

    def definir_escala_pixel(self, distancia_real_metros, pontos_referencia=None):
        """
        Calcula a escala (m/px) baseada em uma distância real conhecida.
        """
        print(f"\n--- CALIBRAÇÃO DE ESCALA (Ref: {distancia_real_metros}m) ---")
        pontos = []

        # MODO 1: Automático
        if pontos_referencia is not None:
            print("Usando coordenadas de referência pré-definidas.")
            pontos = pontos_referencia
        
        # MODO 2: Manual
        else:
            print("Selecione INÍCIO e FIM do objeto de referência na imagem.")
            try:
                fig, ax = plt.subplots(figsize=(12, 8))
                ax.imshow(self.img_rgb)
                ax.set_title(f"Clique em 2 pontos ({distancia_real_metros}m). ENTER para confirmar.")
                pontos = plt.ginput(n=2, timeout=-1)
                plt.close()
                
                if len(pontos) == 2:
                    print("\n" + "="*50)
                    print("Copie a lista abaixo:")
                    lista_fmt = [(int(x), int(y)) for x, y in pontos]
                    print(f"pontos_escala = {lista_fmt}")
                    print("="*50 + "\n")
                else:
                    print("Erro: Selecione exatamente 2 pontos.")
                    return
            except Exception as e:
                print(f"Erro na interface gráfica: {e}")
                return

        # Cálculo
        p1, p2 = pontos
        dist_pixels = math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
        
        if dist_pixels == 0:
            print("Erro: Distância em pixels é zero.")
            return

        self.pixel_scale_m = distancia_real_metros / dist_pixels
        print(f"Distância Imagem: {dist_pixels:.2f} px")
        print(f"Escala Definida: 1 px = {self.pixel_scale_m:.5f} metros")

    def criar_mascara_pilha(self, pontos_fixos=None):
        print("\n--- DELIMITAÇÃO DA PILHA ---")
        pontos = []

        if pontos_fixos is not None:
            print("Usando coordenadas pré-definidas (Modo Automático).")
            pontos = pontos_fixos
        else:
            print("Nenhuma coordenada informada. Abrindo seleção manual...")
            try:
                fig, ax = plt.subplots(figsize=(12, 8))
                ax.imshow(self.img_rgb)
                ax.set_title("Clique para contornar. ENTER para finalizar.")
                pontos = plt.ginput(n=-1, timeout=-1)
                plt.close()
                
                print("\n" + "="*50)
                print("Copie a lista abaixo:")
                lista_fmt = [(int(x), int(y)) for x, y in pontos]
                print(f"pontos_pilha = {lista_fmt}")
                print("="*50 + "\n")
            except Exception as e:
                print(e); return

        if len(pontos) < 3:
            print("Erro: Polígono inválido.")
            return
            
        h, w = self.depth_map.shape
        mask_img = np.zeros((h, w), dtype=np.uint8)
        pts_np = np.array(pontos, dtype=np.int32)
        cv2.fillPoly(mask_img, [pts_np], 255)
        self.mask = mask_img > 0

        masked_depth = self.depth_map.copy()
        masked_depth[~self.mask] = 0
        plt.figure(figsize=(10, 5))
        plt.imshow(masked_depth, cmap='terrain')
        plt.title("Área Isolada")
        plt.show()

    def calcular_volume_calibrado(self, altura_real_max=None):
        if self.depth_map is None or self.mask is None:
            print("ERRO: Gere o mapa de profundidade e a máscara primeiro.")
            return

        if self.pixel_scale_m is None:
            print("AVISO: Escala não definida! Usando input manual.")
            try:
                s_input = input("Tamanho do pixel (m)? [Padrão: 0.05]: ")
                self.pixel_scale_m = float(s_input) if s_input else 0.05
            except: self.pixel_scale_m = 0.05
        
        pixel_area_m2 = self.pixel_scale_m ** 2
        
        pilha_vals = self.depth_map[self.mask]
        pilha_norm = pilha_vals - pilha_vals.min()
        peak_norm = pilha_vals.max() - pilha_vals.min()
        
        print(f"\n--- CÁLCULO FINAL ---")
        if altura_real_max is None:
            try:
                h_input = input(f"Altura MÁXIMA da pilha (m)? [Padrão: 5.0]: ")
                altura_real_max = float(h_input) if h_input else 5.0
            except: altura_real_max = 5.0

        if peak_norm > 0:
            height_scale = altura_real_max / peak_norm
        else: height_scale = 0
            
        pilha_heights_m = pilha_norm * height_scale
        volume_m3 = np.sum(pilha_heights_m * pixel_area_m2)
        area_base = np.sum(self.mask) * pixel_area_m2

        print("="*40)
        print(f"Escala Usada: {self.pixel_scale_m:.5f} m/px")
        print(f"Área Base: {area_base:.2f} m²")
        print(f"Altura Max: {altura_real_max:.2f} m")
        print(f"Volume: {volume_m3:.2f} m³")
        print(f"Massa (2.98 t/m³): {volume_m3 * 2.98:.2f} ton")
        print("="*40)

In [20]:
# --- INPUTS ---
# 1. Objeto de Referência (Comprimento Esteira da Escavadeira)
TAMANHO_REAL_OBJETO = 4.25 
PONTOS_ESCALA = [(883, 316), (1106, 310)]      

# 2. Delimitação da Pilha
PONTOS_PILHA = [(366, 321), (209, 400), (119, 604), (250, 749), (441, 846), 
                (670, 921), (905, 948), (1126, 846), (1239, 679), (1301, 563), 
                (1271, 415), (1162, 317), (1053, 227), (860, 156), (728, 156), (496, 240)]

# --- PIPELINE ---
try:
    # 1. Inicializa
    estimator = DeepVolumeEstimator(CAMINHO_IMAGEM)
    
    # 2. Define Escala
    estimator.definir_escala_pixel(TAMANHO_REAL_OBJETO, pontos_referencia=PONTOS_ESCALA)
    
    # 3. Gera Mapa
    estimator.gerar_mapa_profundidade()
    
    # 4. Aplica Máscara
    estimator.criar_mascara_pilha(pontos_fixos=PONTOS_PILHA)
    
    # 5. Calcula Volume
    estimator.calcular_volume_calibrado() # Se quiser passar altura fixa: calcular_volume_calibrado(altura_real_max=5.0)

except Exception as e:
    print(f"Erro na execução: {e}")

Carregando modelo Depth Anything V2... (aguarde)
Usando dispositivo: CPU


Device set to use cpu



--- CALIBRAÇÃO DE ESCALA (Ref: 4.25m) ---
Usando coordenadas de referência pré-definidas.
Distância Imagem: 223.08 px
Escala Definida: 1 px = 0.01905 metros
Gerando mapa de profundidade...

--- DELIMITAÇÃO DA PILHA ---
Usando coordenadas pré-definidas (Modo Automático).

--- CÁLCULO FINAL ---


Altura MÁXIMA da pilha (m)? [Padrão: 5.0]:  5.0


Escala Usada: 0.01905 m/px
Área Base: 239.98 m²
Altura Max: 5.00 m
Volume: 587.64 m³
Massa (2.98 t/m³): 1751.18 ton
